In [41]:
import numpy as np
import pandas as pd
import json
import matplotlib

In [49]:
with open('/home/kimino/soft/qcm-detector/results.json', 'r') as results_file:
    r_json = json.loads(results_file.read())

In [50]:
def get_tf_positives(experiment, experiment_name):
    true_positives = experiment['True Positives']
    false_positives = [exp for exp in experiment['Detections'] if exp not in experiment['True Positives']]
    tp_df = pd.DataFrame(true_positives)
    fp_df = pd.DataFrame(false_positives)
    tp_df['is_correct'] = True
    fp_df['is_correct'] = False
    df = pd.concat([tp_df, fp_df], ignore_index= True)
    df['experiment'] = experiment_name
    df['path'] = experiment['Path']
    return df
def get_all_detections(experiment_json):
    detection_df = pd.DataFrame()
    for experiment in experiment_json:
        tf_df = get_tf_positives(experiment_json[experiment], experiment)
        detection_df = pd.concat([detection_df, tf_df], ignore_index= True, sort = False)
    return detection_df

In [51]:
results_df = get_all_detections(r_json)
len(np.unique(results_df['experiment']))

ipykernel_launcher:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



23

In [52]:
results_df.groupby(['detector', 'is_correct']).count()

timestamp  frequency  resistance  period  time  \
detector        is_correct                                                   
Expo-predict    False               4          4           4       4     4   
                True               14         14          14      14    14   
IsolationForest False             568        568         568     568   568   
                True              489        489         489     489   489   
Mean-shift      True               30         30          30      30    30   
OneClassSVM     False             245        245         245     245   245   
                True               50         50          50      50    50   

                            severity  notified  experiment  path  
detector        is_correct                                        
Expo-predict    False              4         4           4     4  
                True              14        14          14    14  
IsolationForest False            568       568         568   568  
                True             489       489         489   489  
Mean-shift      True              30        30          30    30  
OneClassSVM     False            245       245         245   245  
                True              50        50          50    50

In [53]:
results_df[results_df['detector'].isin(['Expo-predict', 'Mean-shift'])].groupby(['experiment','detector', 'is_correct']).count()

timestamp  frequency  resistance  \
experiment        detector     is_correct                                     
Hard-1ppm         Expo-predict True                2          2           2   
Hard-3ppm         Expo-predict True                2          2           2   
                  Mean-shift   True                2          2           2   
Hard-5ppm         Expo-predict True                2          2           2   
                  Mean-shift   True                2          2           2   
Hard-5ppm-1       Expo-predict True                1          1           1   
                  Mean-shift   True                3          3           3   
Hard-H20-Tap      Expo-predict True                1          1           1   
                  Mean-shift   True                2          2           2   
Hard-Tap-H20      Expo-predict True                1          1           1   
                  Mean-shift   True                2          2           2   
Hard-Water-1      Expo-predict False               1          1           1   
Hard-Water-3      Expo-predict False               1          1           1   
Hard-Water-4      Expo-predict False               1          1           1   
Hard-Water-5      Expo-predict False               1          1           1   
Insertion-100ppm  Expo-predict True                1          1           1   
                  Mean-shift   True                2          2           2   
Insertion-10ppm   Mean-shift   True                2          2           2   
Insertion-1ppm    Mean-shift   True                1          1           1   
Insertion-30ppm   Mean-shift   True                2          2           2   
Insertion-3ppm    Mean-shift   True                1          1           1   
Insertion-50ppm   Expo-predict True                1          1           1   
                  Mean-shift   True                2          2           2   
Insertion-5ppm    Mean-shift   True                2          2           2   
Overnight-10ppm   Expo-predict True                1          1           1   
                  Mean-shift   True                3          3           3   
Swell-10ppm       Expo-predict True                1          1           1   
                  Mean-shift   True                2          2           2   
Swell-10ppm-after Expo-predict True                1          1           1   
                  Mean-shift   True                2          2           2   

                                           period  time  severity  notified  \
experiment        detector     is_correct                                     
Hard-1ppm         Expo-predict True             2     2         2         2   
Hard-3ppm         Expo-predict True             2     2         2         2   
                  Mean-shift   True             2     2         2         2   
Hard-5ppm         Expo-predict True             2     2         2         2   
                  Mean-shift   True             2     2         2         2   
Hard-5ppm-1       Expo-predict True             1     1         1         1   
                  Mean-shift   True             3     3         3         3   
Hard-H20-Tap      Expo-predict True             1     1         1         1   
                  Mean-shift   True             2     2         2         2   
Hard-Tap-H20      Expo-predict True             1     1         1         1   
                  Mean-shift   True             2     2         2         2   
Hard-Water-1      Expo-predict False            1     1         1         1   
Hard-Water-3      Expo-predict False            1     1         1         1   
Hard-Water-4      Expo-predict False            1     1         1         1   
Hard-Water-5      Expo-predict False            1     1         1         1   
Insertion-100ppm  Expo-predict True             1     1         1         1   
                  Mean-shift   True             2     2         2         2   
Insertion-10p

In [54]:
a = [exp for exp in  list(r_json.keys()) if not exp in list(results_df[results_df['detector'].isin(['Expo-predict', 'Mean-shift'])].groupby(['experiment']).count().index)]
a

['Long-Insertion-10ppm',
 'Hard-0.5ppm_chosen',
 'Long-Insertion-5ppm',
 'Hard-0',
 'Long-Insertion-10ppm-1',
 'Hard-Water-2']

In [55]:
times = np.sum([experiment['Total Detection Time'] for experiment in experiments])
times

2119.701063156128

In [9]:
with open('/home/kimino/soft/qcm-detector/results.json', 'r') as results_file:
    r_json = json.loads(results_file.read())
experiments = [r_json[exp] for exp in r_json]

In [40]:
times = np.sum([experiment['Total Detection Time'] for experiment in experiments])
times

2119.701063156128

In [42]:
results_df = get_all_detections(r_json)


ipykernel_launcher:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [45]:
results_df.groupby(['detector']).count()

,frequency,is_correct,period,resistance,severity,time,timestamp,experiment,path
detector,,,,,,,,,
Expo-predict,16,16,16,16,16,16,16,16,16
Mean-shift,25,25,25,25,25,25,25,25,25


In [14]:
exp_df = pd.DataFrame(experiments)

In [16]:
input_file_path = '/home/kimino/soft/qcm-detector/data/tests/test_descriptions.json'
with open(input_file_path, 'r') as test_json_file:
    input_file = json.loads(test_json_file.read())
tests = input_file['Tests']

In [56]:
for test in tests:
    if 'Long' not in list(test.keys())[0] and 'long' not in list(test.keys())[0]:
        print(list(test.keys())[0], list(test.keys())[0] in results_df['experiment'][results_df['detector'] == 'Expo-predict'].unique() )

Insertion-1ppm True
Hard-Water-4 False
Hard-0.5ppm_chosen False
Hard-1ppm False
Swell-10ppm True
Insertion-5ppm True
Insertion-30ppm True
Hard-H20-Tap True
Hard-5ppm True
Hard-Water-5 False
Hard-5ppm-1 True
Insertion-50ppm True
Hard-Tap-H20 True
Insertion-10ppm True
Hard-0 False
Swell-10ppm-after True
Hard-Water-1 False
Overnight-10ppm True
Hard-Water-3 False
Hard-3ppm True
Insertion-3ppm True
Insertion-100ppm True
Hard-Water-2 False


array(['Insertion-1ppm', 'Hard-Water-4', 'Hard-0', 'Hard-1ppm',
       'Swell-10ppm', 'Insertion-5ppm', 'Insertion-30ppm', 'Hard-H20-Tap',
       'Hard-5ppm', 'Hard-Water-5', 'Hard-5ppm-1', 'Insertion-50ppm',
       'Hard-Tap-H20', 'Insertion-10ppm', 'Hard-Water-1', 'Hard-Water-3',
       'Hard-3ppm', 'Insertion-3ppm', 'Insertion-100ppm', 'Hard-Water-2'],
      dtype=object)